## adjust countries to be the same information

In [1]:
%pip install geopy translate joblib

Note: you may need to restart the kernel to use updated packages.


### Reading files automatically

In [19]:
import os
import pandas as pd

def _adjust_name(name):
    if not('df_' in name):
        return 'df_' + name
    return name
def _get_files_from(base_path = '..\\data\\processed', extension='csv'):
    return [(_adjust_name(name.split('.')[0].lower()), str(os.path.join(root, name))) for (root, _, files) in os.walk(base_path, topdown=False) for name in files if (extension in name)]

def read_files(_from='..\\data\\processed', extension='csv'):
    vars = []
    for item in _get_files_from(_from, extension=extension):
        exec(f'vars.append("{item[0]}")')
        exec(f"{item[0]} = pd.read_csv(r'{item[1]}', sep=';', decimal=',')")
    return vars

# _get_files_from()

## Load data

In [115]:
import os
import pandas as pd
import numpy as np
import joblib
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

base_path = '..\\data\\processed'

df_noaa_global = pd.read_csv(r'..\\data\\processed\\noaa_global\\noaa_global_final.csv', sep=';', decimal=',')
df_vinhos = pd.read_csv(r'..\\data\\processed\\tech_challenge\\df_vinhos.csv', sep=';', decimal=',')
df_temperature_change_data = pd.read_csv(r'..\\data\\processed\\temp_change\\temperature_change_Data.csv', sep=';', decimal=',')
df_wbpy = pd.read_csv(r'..\\data\\processed\\wbpy\\wbpy.csv', sep=';', decimal=',')
df_clima_rs =  pd.read_csv(r'..\\data\\processed\\inmet\\rs_final.csv', sep=';', decimal=',')

## Dados climáticos RS

In [116]:
df_precp_global = df_noaa_global.loc[df_noaa_global['stat']=='PRCP']
df_precp_global = df_precp_global[['year','country_code','value_median']]
df_precp_global = df_precp_global.loc[df_precp_global.year >= min(df_clima_rs.year)]
df_precp_global = df_precp_global.loc[df_precp_global.year <= max(df_clima_rs.year)]
df_clima_rs = df_clima_rs.rename(columns={'PRCP':'value_median'})
df_clima_rs['country_code'] = 'BR-RS'
df_full = pd.concat([df_precp_global, df_clima_rs[['year','country_code','value_median']]])

In [117]:
def _mape(real, comparado):
    return list(abs(np.array(real) - comparado)/comparado)

_max_precip = max(df_clima_rs['value_median'])
_min_precip = min(df_clima_rs['value_median'])
_mean_precip = np.mean(df_clima_rs['value_median'])

df_full['dif_perc_max'] = _mape(df_full['value_median'], _max_precip)
df_full['rank_per_max_year'] = df_full.groupby('year')['dif_perc_max'].rank()
final_rank_max = df_full.groupby(['country_code']).agg({'rank_per_max_year':'sum'}).reset_index()
final_rank_max['final_rank_max'] = final_rank_max['rank_per_max_year'].rank()
df_full = df_full.drop(columns=['rank_per_max_year']).merge(final_rank_max.drop(columns=['rank_per_max_year']))

df_full['dif_perc_min'] = _mape(df_full['value_median'], _min_precip)
df_full['rank_per_min_year'] = df_full.groupby('year')['dif_perc_min'].rank()
final_rank_max = df_full.groupby(['country_code']).agg({'rank_per_min_year':'sum'}).reset_index()
final_rank_max['final_rank_min'] = final_rank_max['rank_per_min_year'].rank()
df_full = df_full.drop(columns=['rank_per_min_year']).merge(final_rank_max.drop(columns=['rank_per_min_year']))

df_full['dif_perc_mean'] = _mape(df_full['value_median'], _mean_precip)
df_full['rank_per_mean_year'] = df_full.groupby('year')['dif_perc_mean'].rank()
final_rank_max = df_full.groupby(['country_code']).agg({'rank_per_mean_year':'sum'}).reset_index()
final_rank_max['final_rank_mean'] = final_rank_max['rank_per_mean_year'].rank()
df_full = df_full.drop(columns=['rank_per_mean_year']).merge(final_rank_max.drop(columns=['rank_per_mean_year']))

df_full = df_full.query('final_rank_max <= 6 or final_rank_min <= 6 or final_rank_mean <= 6')

In [118]:

fig = go.Figure()

fig = px.line(
    df_full, 
    x='year', y='value_median', color='country_code'
    , custom_data=['year', 'value_median']
)

fig.show()

In [38]:
df_full.sort_values(['year','value_median'])

,year,country_code,value_median
0,1971,AF,0.0
1,1971,AO,0.0
2,1971,AR,0.0
3,1971,AT,0.0
4,1971,AU,0.0
...,...,...,...
11,2018,BR-RS,38.0
12,2019,BR-RS,34.4
13,2020,BR-RS,22.2
14,2021,BR-RS,22.6
